[Tutorial URL](https://python.langchain.com/docs/get_started/quickstart)


In [1]:
!pipenv install langchain-openai
!pipenv install beautifulsoup4
!pipenv install langchain-community
!pipenv install faiss-cpu
!pipenv install langchain

Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing langchain-openai...
Resolving langchain-openai...
✔ Installation Succeeded
⠋ Installing langchain-openai...
Installing dependencies from Pipfile.lock (f18864)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set 

In [2]:
# pipenv loads the environment variables from the .env file
# get OPENAI_API_KEY from the .env file
# type: ignore
OPENAI_API_KEY = %env OPENAI_API_KEY

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [4]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith can help with testing in the following ways:\n\n1. Test Case Generation: Langsmith can generate test cases automatically based on the specifications provided. It uses techniques like symbolic execution and constraint solving to generate a wide range of test cases, covering different execution paths and edge cases.\n\n2. Test Automation: Langsmith can automate the execution of test cases by generating code that interacts with the system under test. This eliminates the need for manual test execution, saving time and effort.\n\n3. Test Oracles: Langsmith can also automatically generate test oracles, which are mechanisms to determine the correctness of the system's output. It can compare the expected output with the actual output and identify any deviations or errors.\n\n4. Test Coverage Analysis: Langsmith can analyze the coverage of the generated test cases, indicating which parts of the code or system are exercised by the tests and which parts are not. This 

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are world class technical documentation writer.'),
    ('user', '{input}')
])


In [6]:
chain = prompt | llm
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith can assist with testing in several ways:\n\n1. Test Case Documentation: Langsmith can generate detailed test case documentation based on the specifications provided. It ensures that all possible scenarios and edge cases are covered in the testing process.\n\n2. Test Data Generation: Langsmith can automatically generate test data sets, including both valid and invalid data, for thorough testing. This helps uncover hidden bugs and ensures comprehensive test coverage.\n\n3. Test Script Generation: Langsmith can generate test scripts or test code snippets in various programming languages, based on the specified test cases. This saves time and effort for testers, allowing them to focus on executing tests rather than writing code.\n\n4. Test Automation: Langsmith can assist in automating the testing process by generating automation scripts or integrating with existing test automation frameworks. This accelerates the testing cycle and improves efficiency.\n\n5. Te

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can greatly help with testing by providing a powerful tool for automating the testing process. Here are a few ways in which Langsmith can assist with testing:\n\n1. Test Case Generation: Langsmith can generate test cases automatically based on predefined specifications or requirements. This saves time and effort compared to manually writing test cases, especially for complex systems.\n\n2. Test Execution: Langsmith can execute test cases and validate the expected outcomes. It can handle both functional and non-functional testing, such as performance, security, and usability testing.\n\n3. Test Data Management: Langsmith can generate and manage test data, ensuring that the tests cover a wide range of scenarios and edge cases. It can also handle data-driven testing, where tests are executed with different sets of input data.\n\n4. Test Reporting: Langsmith can generate detailed reports after test execution, providing insights into the test coverage, pass/fail status, and any d

In [10]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [11]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [12]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
vector

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
                                          <context>
                                          {context}
                                          </context>

                                          Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [15]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response['answer'])

LangSmith can help with testing in several ways. It allows users to construct datasets and use them to test changes to prompts or chains. Users can run the chain over the data points and visualize the outputs, and they can easily assign feedback to runs and mark them as correct or incorrect. LangSmith also provides evaluators that can be used to evaluate the results of the test runs, although they may not be perfect. Additionally, LangSmith offers annotation queues that allow users to manually review and annotate runs, assessing subjective qualities and validating auto-evaluated runs.
